## Model Training

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("data/gemstone.csv")
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...
19994,19994,1.64,Premium,G,SI1,61.2,58.0,7.63,7.69,4.67,11480
19995,19995,0.90,Ideal,F,SI1,62.5,56.0,6.12,6.15,3.83,4182
19996,19996,0.70,Ideal,G,VS1,61.5,55.0,5.72,5.74,3.52,3027
19997,19997,1.23,Very Good,H,VS2,61.5,59.0,6.88,6.92,4.24,6473


In [3]:
df=df.drop(labels=['id'],axis=1)

In [4]:
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
19994,1.64,Premium,G,SI1,61.2,58.0,7.63,7.69,4.67,11480
19995,0.90,Ideal,F,SI1,62.5,56.0,6.12,6.15,3.83,4182
19996,0.70,Ideal,G,VS1,61.5,55.0,5.72,5.74,3.52,3027
19997,1.23,Very Good,H,VS2,61.5,59.0,6.88,6.92,4.24,6473


In [5]:
# Indepenent and Dependent features
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [6]:
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [7]:
# defining custom ranking for each ordinal variable
cut_categories=["Fair","Good","Very Good","Premium","Ideal"]
clarity_categories=["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"] 
color_categories=["D","E","F","G","H","I","J"]

In [8]:
from sklearn.impute import SimpleImputer            #handle missing values
from sklearn.preprocessing import StandardScaler   #handle feature scaling
from sklearn.preprocessing import OrdinalEncoder   # if categorical data has ranks in it we use ordinal encoding otherwise OneHotEncoding
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

### numerical pipeline

In [9]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

### categorical pipeline

In [10]:
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

### joining numerical and categorical pipeline

In [11]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

### Train-Test split

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [13]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())

In [14]:
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [15]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.821882,0.544531,0.393540,-0.861823,-0.898640,-0.835740,-0.128424,0.290999,0.679096
1,-0.180653,-0.935552,-0.644114,0.042581,0.002565,-0.054944,-1.131669,2.137923,0.016102
2,-0.608139,-0.658036,-0.644114,-0.512598,-0.547170,-0.575474,-1.131669,0.906640,-0.646891
3,0.225458,1.654593,-0.644114,0.400761,0.363047,0.537883,-1.131669,0.290999,1.342089
4,-1.099747,0.267015,-0.125287,-1.363274,-1.340231,-1.327352,-1.131669,2.137923,0.679096


### model Training

In [16]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [18]:
regression.coef_

array([[ 6369.43454923,  -118.9560519 ,   -67.38048629, -2040.77195721,
          192.26066796,  -325.06234318,    65.33610729,  -459.07562026,
          649.36991254]])

In [19]:
regression.intercept_

array([4003.83441674])

In [20]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

### Training multiple models

In [21]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
model_list=[]
r2_list=[]


for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    #making prediction
    y_pred=model.predict(X_test)
    mae,rmse,r2_square=evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model Training Performance")
    print("RMSE: ",rmse)
    print("MAE: ",mae)
    print("R2_SCORE: ",r2_square*100)
    r2_list.append(r2_score)
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE:  1063.7704695291059
MAE:  701.455060799888
R2_SCORE:  93.48355957345508


Lasso
Model Training Performance
RMSE:  1064.0631809540946
MAE:  702.1731124301589
R2_SCORE:  93.47997289936181


Ridge
Model Training Performance
RMSE:  1063.8070454625488
MAE:  701.6590987132645
R2_SCORE:  93.48311145236939


ElasticNet
Model Training Performance
RMSE:  1593.4526329696935
MAE:  1096.7067902780439
R2_SCORE:  85.37846778457609


